## Data Extraction scripts

In [4]:
import pandas as pd
import sqlite3
import gzip

In [5]:
# Helper functions
def age_group(n):
    # 10 year age buckets
    if not n:
        return
    elif n == "> 89":
        return n
    n = int(n) // 10
    return f"{n}0-{n}9"

def icd_blocks(code):
    # Map diagnosis code to broader condition
    # source: http://www.icd9data.com/2015/Volume1/default.htm
    if not code:
        return "Unknown"
    code = code.split(",")[0].split(".")[0]
    if code[0].isalpha():
        if code.startswith("E") or code.startswith("V"):
            return "Supplementary Classification"
        else:
            return "Unknown"
    code = int(code)
    if code <= 139:
        return "Infectious and Parasitic Diseases"
    elif code <= 239:
        return "Neoplasms"
    elif code <= 279:
        return "Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders"
    elif code <= 289:
        return "Diseases of the Blood and Blood-forming Organs"
    elif code <= 319:
        return "Mental Disorders"
    elif code <= 389:
        return "Diseases of the Nervous System and Sense Organs"
    elif code <= 459:
        return "Diseases of the Circulatory System"
    elif code <= 519:
        return "Diseases of the Respiratory System"
    elif code <= 579:
        return "Diseases of the Digestive System"
    elif code <= 629:
        return "Diseases of the Genitourinary System"
    elif code <= 679:
        return "Complications of Pregnancy, Childbirth, and the Puerperium"
    elif code <= 709:
        return "Diseases of the Skin and Subcutaneous Tissue"
    elif code <= 739:
        return "Diseases of the Musculoskeletal System and Connective Tissue"
    elif code <= 759:
        return "Congenital Anomalies"
    elif code <= 779:
        return "Certain Conditions originating in the Perinatal Period"
    elif code <= 799:
        return "Symptoms, Signs and Ill-defined Conditions"
    else:
        return "Injury and Poisoning"
    
def get_primary_diagnosis(patientunitstayid, diagnosis_df):
    if patientunitstayid not in diagnosis_df["patientunitstayid"]:
        return
    return diagnosis_df[
        (diagnosis_df["patientunitstayid"] == patientunitstayid) & 
        (diagnosis_df["diagnosispriority"] == "Primary")
    ]["category"].iloc[0]

In [9]:
# after extracting the gz sqlite3 file:
conn = sqlite3.connect("eicu_v2_0_1.sqlite3")

def table_to_df(table):
    query = f"SELECT * FROM {table}"
    return pd.read_sql_query(query, conn)

# get tables of interest
admissiondx = table_to_df("admissiondx")
apachepatientresult = table_to_df("apachepatientresult")
hospital = table_to_df("hospital")
patient = table_to_df("patient")
diagnosis = table_to_df("diagnosis")

conn.close()

# Add diagnosis category
diagnosis["category"] = diagnosis.apply(lambda row:icd_blocks(row["icd9code"]), axis=1)

# merging patient dataframes
# admissiondx
admissiondx = admissiondx[["patientunitstayid","admitdxenteredoffset"]].drop_duplicates()
patient = patient.join(admissiondx.set_index("patientunitstayid"), on="patientunitstayid", how="left")
# hospital
hospital["hospitalid"] = hospital["hospitalid"].astype(int)
patient["hospitalid"] = patient["hospitalid"].astype(int)
patient = patient.join(hospital.set_index("hospitalid"), on="hospitalid", how="left")
# apache result
apachepatientresult = apachepatientresult[apachepatientresult["apacheversion"]=="IVa"]
patient = patient.join(
    apachepatientresult.set_index("patientunitstayid"), 
    on="patientunitstayid", how="left"
)

# Add additional variables of interest
patient["agegroup"] = patient.apply(lambda row:age_group(row["age"]), axis=1)
patient["primarydiagnosis"] = patient.apply(
    lambda row:get_primary_diagnosis(row["patientunitstayid"], diagnosis), axis=1
)
patient = patient.replace({"age": {"> 89": "90"}})

# keeping only columns of interest
patient_df = patient[[
    # ids: ICU stay id, hospital stay id, unique patient id
    # each patient have >= 1 hospital stays, each hospital stay have >= 1 icu stays
    'patientunitstayid', 'patienthealthsystemstayid', 'uniquepid',
    # admit time and lengths
    'hospitaladmittime24', 'hospitaldischargetime24', 
    'unitadmittime24', 'unitdischargetime24', 'unitvisitnumber', 
    # all of the following are w.r.t. unit admit time which is 0
    'hospitaladmitoffset', 'unitdischargeoffset', 'hospitaldischargeoffset',
    'admitdxenteredoffset',
    # demographics
    'gender', 'age', 'agegroup', 'ethnicity', 'admissionheight', 'admissionweight',
    'primarydiagnosis',
    # hospital
    'hospitalid', 'wardid', 'numbedscategory', 'region',
    # apache
    'acutephysiologyscore', 'apachescore',
    'predictedicumortality', 'actualicumortality',
    'predictediculos', 'actualiculos', 
    'predictedhospitalmortality','actualhospitalmortality', 
    'predictedhospitallos', 'actualhospitallos',
    
]]
print(patient_df.shape)

DatabaseError: Execution failed on sql 'SELECT * FROM admissiondx': no such table: admissiondx

In [4]:
# save dfs of interest
diagnosis.to_csv("diagnosis.csv", index=False)
patient_df.to_csv("patient.csv", index=False)
